In [1]:
# Importation
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# **I - Data management**

In [ ]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200.csv")

In [ ]:
df = pd.read_csv("C:/Users/Yanni/OneDrive/Desktop/projet d4g/scores_pour_annotation.csv") # Pour moi pcq je galere

## Présentation des données 


Le jeu de données qu'on traite dans le notebook est issue du jeu de données open source  [La fiscalité et les dépenses publiques](https://www.data.gouv.fr/fr/datasets/r/bc085888-e6bd-445d-b3f4-632190c29e3f). Celui-ci est notament constitué des réponses textuelles des contribuables à des questions portant sur la fiscalité en France. Dans cette étude, on se penche principalement sur la question suivante  : *Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?*.

Pour extraire idées majeures des réponses textuelles, on utilise le modèle de langue ollama : *hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4*. Avec les hyperparamètres **top_p = 0.95**
et **temperature = 0**.

On donne à ce LLM le prompt suivant :


[prompt]
system = """
But: extraire les idées principales DISTINCTES d’un texte pour analyse.

Règles:
1. N’utiliser QUE le contenu entre <<< TEXT >>>.
2. Extraire la liste des idées DISTINCTES et PRINCIPALES.
   - Chaque idée = une phrase claire, autonome, reformulée si nécessaire.
3. Pour CHAQUE idée, annoter:
   - type: "statement" (constat) OU "proposition" (suggestion/recommandation/objectif).
   - syntax: "negative" si la phrase contient une négation explicite (ex.: "ne", "n'", "ne pas", "ne plus", "non"), sinon "positive".
   - semantic: "positive", "negative" ou "neutral" (valence sémantique).
4. Sortie STRICTEMENT en CSV avec entête EXACTE:
   CSV:description,type,syntax,semantic
   - Délimiteur: virgule.
   - Chaque description entre guillemets doubles.
   - Échapper tout guillemet interne par duplication (ex.: ""chat"").
   - NE RIEN AJOUTER d’autre (pas de texte avant/après, pas de code fences).
   - Pas de lignes vides.

Exemple:
CSV:description,type,syntax,semantic
"Les chats retombent sur leurs pattes",statement,positive,neutral
"Les chats n'ont pas neuf vies",statement,negative,negative
"Il faut mieux prendre soin des chats pour prolonger leur vie",proposition,positive,positive
"""
user = """<<< {input} >>>"""

Ainsi, on construit le jeu de données **df**. Ces variables sont les suivantes :

- **authorId** : Identifiant de l’auteur
- **contrib_index** : Numéro de la contribution
- **contribution** : Texte original de l'auteur
- **C** : Score calculé par la métrique *QualIT*
- **n_ideas** : Nombre d’idées extraites
- **len_contrib** : Longueur du texte original
- **Matthias, Yannis, Garance** : Note allant de 0 à 10 évaluant selon la personne la qualité de l'extraction
- **Hallucinations** : Présence d'hallucination dans l'extraction
- **Idées_non_ind** : Idées dénuée de sens dans l'extraction

On construit ensuite de nouvelles variables à partir de ces données brutes pour faciliter l'analyse :

In [3]:
# Calcul du score humain moyen
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1)

# Copie des colonnes d'hallucinations/d'idées invalides en version catégorielle
mapping = {0:"No", 1:"Yes"}
df["pres_hallu"] = df["Hallucinations"].map(mapping)
df["pres_idees_inv"] = df["Idées_non_ind"].map(mapping)

# Nombre de tokens par contribution et par extractions (comptage simple par espaces)
df["nb_tokens_contrib"] = df["contribution"].apply(lambda x: len(str(x).split()))
df["nb_tokens_extraction"] = df["ideas_text"].apply(lambda x: len(str(x).split()))

# **II - Validité de la notation et de la métrique**

In [ ]:
# Calcul des corrélations de Pearson entre chaque paire d'évaluateurs
corr_GM = df["Garance"].corr(df["Matthias"], method='pearson')
corr_GY = df["Garance"].corr(df["Yannis"], method='pearson')
corr_MY = df["Matthias"].corr(df["Yannis"], method='pearson')
# Corrélation de Pearson entre score humain et score QualIT
correlation = df["score_humain"].corr(df["C"], method='pearson')

print(f"Corrélation (de Pearson) entre Garance et Matthias : {corr_GM:.2f}")
print(f"Corrélation (de Pearson) entre Garance et Yannis : {corr_GY:.2f}")
print(f"Corrélation (de Pearson) entre Matthias et Yannis : {corr_MY:.2f}")
print(f"Corrélation entre le score humain moyen et le score QualIT : {correlation:.2f}")

Les trois évaluateurs humains (Matthias, Yannis, Garance) semblent globalement en accord.

La métrique QualIT et les notes humaines sont assez fortement corrélées, bien que moins que les notes entre évaluateurs humains. La métrique semble donc assez fiable pour évaluer la qualité des extractions. Nous pouvons toutefois observer quelques visualisations afin d'identifier cette baisse de corrélation.

# **III - Visualisations**

### **A - Caractéristiques des contributions**

In [ ]:
# Barplot du nombre de charactères par contribution sur les 200 premières contributions
fig = px.histogram(
    df, x="nb_tokens_contrib",
    color_discrete_sequence=["#2a2781"]
)
fig.update_layout(
    title={"text": "<b>Distribution du nombre de tokens par contribution</b>"}, 
    xaxis_title="Nombre de tokens", 
    yaxis_title="Nombre de contributions", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre de tokens = %{x}<br>" \
        "Nombre de contributions = %{y}<extra></extra>"
)
fig.show()

Au vu de la distribution, il semble judicieux de regrouper les contributions en différentes catégories de longueur (arbitraire).

In [4]:
# Répartition des contributions en 5 catégories équilibrées selon le nombre de tokens (basé sur les quantiles)
df["contrib_tokens_bins"] = pd.qcut(df["nb_tokens_contrib"], q=4, labels=[
    "Très court", "Court", "Moyen", "Long"
])

# Autre options : catégories basées sur des seuils fixes
df["contrib_tokens_bins_fixe"] = pd.cut(df["nb_tokens_contrib"], bins=[0, 50, 100, 200, np.inf], labels=[
    "[0-50]", "[51-100]", "[101-200]", "[>201]"
])

### **B - Caractéristiques des extractions**

In [ ]:
# Barplot - bins fixes 
### hallucinations
fig1 = px.histogram(
    df, x="contrib_tokens_bins_fixe", y="Hallucinations", 
    histfunc="avg", color="contrib_tokens_bins_fixe", 
    color_discrete_sequence=["#e585c2", "#5db557", "#057dcd", "#730220"], 
    category_orders={"contrib_tokens_bins_fixe": ["[0-50]", "[51-100]", "[101-200]", "[>201]"]}
)
fig1.update_layout(
    # title={"text": "<b>Hallucinations rate per length category</b>"}, 
    xaxis_title="<b>Length category (in tokens)</b>", yaxis_title="<b>Hallucinations rate</b>",
    width=700, height=450, showlegend=False
)
fig1.update_traces(hovertemplate ="Hallucinations rate = %{y}<extra></extra>")
### Idées invalides
fig2 = px.histogram(
    df, x="contrib_tokens_bins_fixe", y="Idées_non_ind", 
    histfunc="avg", color="contrib_tokens_bins_fixe", 
    color_discrete_sequence=["#e585c2", "#5db557", "#057dcd", "#730220"], 
    category_orders={"contrib_tokens_bins_fixe": ["[0-50]", "[51-100]", "[101-200]", "[>201]"]}
)
fig2.update_layout(
    # title={"text": "<b>Invalid ideas rate per length category</b>"}, 
    xaxis_title="<b>Length category (in tokens)</b>", yaxis_title="<b>Invalid ideas rate</b>",
    width=700, height=450, showlegend=False
)
fig2.update_traces(hovertemplate ="Invalid ideas rate = %{y}<extra></extra>")

# Affichage des figures
fig1.show()
fig2.show()

### **C - Caractéristiques des évaluations**

Attention : Les extractions échouées ne sont pas sur la pluspart des graphiques car elles n'ont pas de valeur dans "Hallucinations" ou "Idées_non_ind".

In [22]:
# Heatmap de densité en arrière-plan
fig = px.density_contour(
    df, x="C", y="score_humain",
    nbinsx=20, nbinsy=20
)
fig.update_traces(contours_coloring="fill", contours_showlabels = False, colorscale="Blues")
# Scatter plot
fig.add_scatter(
    x=df["C"], y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate="QualIT score = %{x}<br>Human score = %{y}<extra></extra>"
)
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    xaxis_title="QualIT score", yaxis_title="Human score",
    width=700, height=500,
    coloraxis_showscale=False
)
fig.show()

In [28]:
# Coloration avec hallucinations - version linéaire
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="pres_hallu", color_discrete_sequence=["#ff6361", "#003f5c"]
)
fig.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Score humain vs métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Présence <br>d'hallucinations",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [ ]:
# Coloration avec hallucinations - version logarithmique
df["log_score_humain"] = np.log10(df["score_humain"]+1)

fig = px.scatter(
    df, x="C", y="log_score_humain", 
    color="pres_hallu", color_discrete_sequence=["#ff6361", "#003f5c"]
)
fig.add_shape(      # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=1, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>log(Score humain + 1) vs métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="log(Score humain + 1)", 
    legend_title="Présence <br>d'hallucinations",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "log(Score humain + 1) = %{y}<extra></extra>"
)
fig.show()

In [ ]:
# Coloration avec idées invalides - version linéaire
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="pres_idees_inv", color_discrete_sequence=["#ff6361", "#003f5c"]
)
fig.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Score humain vs métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Présence d'idées <br>invalides",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [8]:
# Scatter plot - bins fixes
### Hallucinations
fig1 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins_fixe", 
    color_discrete_sequence=["#e585c2", "#5db557", "#45abdb", "#4B0202"],
    symbol="pres_hallu", symbol_map={"Yes":"circle", "No":"x"},
    category_orders={"contrib_tokens_bins_fixe": ["[0-50]", "[51-100]", "[101-200]", "[>201]"]}
)
fig1.for_each_trace(
    lambda trace: trace.update(marker=dict(
        size=8 if trace.name.endswith("Yes") else 6,
        opacity=1.0 if trace.name.endswith("Yes") else 0.5,
        line=dict(width=1) if trace.name.endswith("Yes") else None
    ))
)
fig1.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y", line=dict(color="#d41010")
)
fig1.update_layout(
    # title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="QualIT score", yaxis_title="Human score", 
    legend_title="<b>Length category, <br>Hallucinations presence</b>",
    width=700, height=500
)
fig1.update_traces(
    hovertemplate =
        "QualIT score = %{x}<br>" \
        "Human score = %{y}<extra></extra>"
)
### Idées invalides
fig2 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins_fixe", 
    color_discrete_sequence=["#e585c2", "#5db557", "#45abdb", "#4B0202"],
    symbol="pres_idees_inv", symbol_map={"Yes":"circle", "No":"x"},
    category_orders={"contrib_tokens_bins_fixe": ["[0-50]", "[51-100]", "[101-200]", "[>201]"]}
)
fig2.for_each_trace(
    lambda trace: trace.update(marker=dict(
        size=8 if trace.name.endswith("Yes") else 6,
        opacity=1.0 if trace.name.endswith("Yes") else 0.5,
        line=dict(width=1) if trace.name.endswith("Yes") else None
    ))
)
fig2.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y", line=dict(color="#d41010")
)
fig2.update_layout(
    # title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="QualIT score", yaxis_title="Human score", 
    legend_title="<b>Length category, <br>Invalid ideas presence</b>",
    width=700, height=500
)
fig2.update_traces(
    hovertemplate =
        "QualIT score = %{x}<br>" \
        "Human score = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()

In [71]:
# Calcul des moyennes
df_bar = df.groupby("contrib_tokens_bins_fixe", observed=False)[["C", "score_humain"]].mean().reset_index()
# Mise à l'échelle
df_bar["score_humain"] = df_bar["score_humain"]/10
# Ordre des catégories
order = ["[0-50]", "[51-100]", "[101-200]", "[>201]"]
df_bar["contrib_tokens_bins_fixe"] = pd.Categorical(df_bar["contrib_tokens_bins_fixe"], categories=order, ordered=True)
df_bar = df_bar.sort_values("contrib_tokens_bins_fixe")

# Barplot
fig1 = go.Figure()
fig1.add_trace(go.Bar(
    x=df_bar["contrib_tokens_bins_fixe"], y=df_bar["C"],
    name="QualIT score", marker_color="#ecbba0",
))
fig1.add_trace(go.Bar(
    x=df_bar["contrib_tokens_bins_fixe"], y=df_bar["score_humain"],
    name="Human score", marker_color="#96c6e4",
))
fig1.update_layout(
    barmode="group",
    xaxis_title="<b>Length category (in tokens)</b>",
    yaxis_title="<b>QualIT/human score</b>",
    width=700, height=450
)
fig1.update_traces(hovertemplate="Score = %{y}<extra></extra>")

fig1.show()